# Cognitive Action Training Data Generator

This notebook generates high-quality training data for cognitive action recognition using Ollama locally or via API.

**Based on Scientific Taxonomies:**
- Bloom's Taxonomy (cognitive processes)
- Guilford's Structure of Intellect
- Krathwohl's Affective Domain
- Gross's Emotion Regulation Model
- Metacognitive Process Frameworks

**Target:** 100,000+ diverse examples of explicit cognitive and psychological actions

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install requests pandas numpy tqdm matplotlib seaborn aiohttp nest-asyncio

# Clone the repository
import os
if not os.path.exists('datagen'):
    print("Cloning datagen repository...")
    !git clone https://github.com/ChuloIva/datagen.git
    print("✅ Repository cloned successfully!")
else:
    print("✅ Repository already exists")

# If running locally with Ollama installed:
# !pip install ollama

import json
import time
import random
import asyncio
import aiohttp
import nest_asyncio
import requests
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Apply nest_asyncio to allow nested event loops (required for Jupyter/Colab)
nest_asyncio.apply()

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)

print("Dependencies installed successfully!")
print("✓ nest_asyncio applied - async operations will work in Jupyter/Colab")

## 2. Import Supporting Modules

The following Python modules are part of the repository:
- `variable_pools.py` - Contains cognitive action taxonomies and variable pools
- `prompt_templates.py` - Template system for generating diverse prompts
- `data_generator.py` - Main data generation engine

These files will be imported from the cloned `datagen/` directory.

In [ ]:
# Import variable pools module
import sys
import os

# Add the datagen directory to Python path
datagen_dir = os.path.abspath('datagen')
if datagen_dir not in sys.path:
    sys.path.insert(0, datagen_dir)

# Import from the repository
from variable_pools import *

print("Variable pools loaded successfully!")
print(f"Total cognitive actions: {len(COGNITIVE_ACTIONS)}")
print(f"Total domains: {len(DOMAINS)}")
print(f"Total subjects: {len(SUBJECTS)}")

In [ ]:
# Import prompt templates module
from prompt_templates import *

print("Prompt templates loaded successfully!")
print(f"Single action templates: {len(SINGLE_ACTION_TEMPLATES)}")
print(f"Chain templates: {len(CHAIN_TEMPLATES)}")
print(f"Dialogue templates: {len(DIALOGUE_TEMPLATES)}")
print(f"Thought stream templates: {len(THOUGHT_STREAM_TEMPLATES)}")
print(f"Negative templates: {len(NEGATIVE_TEMPLATES)}")

## 3. Ollama Setup

Choose one of the following options:

### Option A: Local Ollama Installation (Recommended)

If you have Ollama running locally, modify the URL below to point to your instance:

In [ ]:
# ============================================================================
# PROPER OLLAMA SETUP FOR GOOGLE COLAB WITH HIGH PARALLELISM
# ============================================================================
# This must be run BEFORE starting Ollama server for the first time!

import os
import subprocess
import time

# Stop any existing Ollama processes
print("🛑 Stopping any existing Ollama processes...")
subprocess.run(['pkill', '-9', 'ollama'], stderr=subprocess.DEVNULL)
time.sleep(2)

# Set environment variables BEFORE starting Ollama
print("\n⚙️  Configuring Ollama environment variables...")
os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
os.environ['OLLAMA_ORIGINS'] = '*'
os.environ['OLLAMA_NUM_PARALLEL'] = '16'  # Allow 16 parallel context buffers
os.environ['OLLAMA_MAX_QUEUE'] = '512'
os.environ['OLLAMA_MAX_LOADED_MODELS'] = '1'
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia'  # Important for GPU

print("Ollama configuration:")
for key in ['OLLAMA_HOST', 'OLLAMA_NUM_PARALLEL', 'OLLAMA_MAX_QUEUE', 'OLLAMA_MAX_LOADED_MODELS']:
    print(f"  {key}: {os.environ.get(key)}")

# Start Ollama server with environment variables
print("\n🚀 Starting Ollama server...")
subprocess.Popen(['ollama', 'serve'], 
                 env=os.environ.copy(),
                 stdout=subprocess.DEVNULL,
                 stderr=subprocess.DEVNULL)

print("⏳ Waiting for Ollama to start...")
time.sleep(5)

# Verify Ollama is running
try:
    import requests
    response = requests.get('http://localhost:11434/api/tags', timeout=5)
    if response.status_code == 200:
        print("✅ Ollama is running successfully!")
        models = response.json().get('models', [])
        if models:
            print(f"\n📦 Available models: {len(models)}")
            for model in models:
                print(f"  - {model['name']}")
        else:
            print("\n⚠️  No models loaded yet. Pull a model with: !ollama pull gemma3:27b")
    else:
        print("❌ Ollama responded but with error")
except Exception as e:
    print(f"❌ Error connecting to Ollama: {e}")
    print("Try running: !ollama serve")

print("\n" + "="*60)
print("IMPORTANT: Understanding VRAM Usage")
print("="*60)
print("With gemma3:27b (18GB model):")
print("  • Model weights: ~18GB (loaded once)")
print("  • Each parallel context: ~1-2GB")
print("  • 16 parallel = 18GB + ~16-32GB = 34-50GB total")
print("\nYour 18GB usage is NORMAL if:")
print("  ✓ Only 1 model is loaded")
print("  ✓ Not currently processing requests")
print("\nVRAM will increase to ~35-40GB when generation starts!")

In [ ]:
# Configure Ollama for high parallelism
# This tells Ollama to process up to 16 requests simultaneously
# Adjust based on your VRAM (40GB can handle 12-16 parallel for gemma3:27b)

import os

# Set environment variables for Ollama
os.environ['OLLAMA_NUM_PARALLEL'] = '16'  # Max parallel requests per model
os.environ['OLLAMA_MAX_QUEUE'] = '512'     # Max queued requests
os.environ['OLLAMA_MAX_LOADED_MODELS'] = '1'  # Only need 1 model loaded

print("Ollama configuration:")
print(f"  OLLAMA_NUM_PARALLEL: {os.environ.get('OLLAMA_NUM_PARALLEL')}")
print(f"  OLLAMA_MAX_QUEUE: {os.environ.get('OLLAMA_MAX_QUEUE')}")
print(f"  OLLAMA_MAX_LOADED_MODELS: {os.environ.get('OLLAMA_MAX_LOADED_MODELS')}")
print("\n⚠️  IMPORTANT: You must RESTART Ollama for these changes to take effect!")
print("\nIn Colab, restart Ollama with:")
print("  !killall ollama")
print("  !nohup ollama serve > ollama.log 2>&1 &")
print("  time.sleep(5)")
print("\nThen verify with: !curl http://localhost:11434/api/tags")

### ⚙️ Configure Ollama for Maximum Parallelism (IMPORTANT!)

**Run this BEFORE starting generation to maximize GPU utilization:**

By default, Ollama only processes 1-4 requests at a time. We need to increase this to match our `max_parallel` setting.

### 🚀 Performance Optimization: Parallel Batch Processing

This notebook now uses **async parallel processing** for significantly faster generation!

**Key Features:**
- Uses Python's `asyncio` and `aiohttp` for concurrent requests
- Automatically batches multiple requests to Ollama
- Controlled by `max_parallel` parameter (default: 4)

**Configuration Guide:**

| Hardware Setup | Recommended `max_parallel` | Expected Speedup |
|---------------|---------------------------|------------------|
| CPU only, 16GB RAM | 2-4 | 2-4x |
| CPU only, 32GB+ RAM | 4-8 | 4-8x |
| GPU, 8-12GB VRAM | 4-8 | 4-8x |
| GPU, 16-24GB VRAM | 8-16 | 8-16x |
| GPU, 24GB+ VRAM | 16-32 | 16-32x |

**Environment Variables (Optional):**
```bash
# Set these before starting Ollama for optimal performance
export OLLAMA_NUM_PARALLEL=8    # Max parallel requests per model
export OLLAMA_MAX_QUEUE=512     # Max queued requests
export OLLAMA_MAX_LOADED_MODELS=3  # Max models in memory
```

**Usage:**
```python
# Create generator with custom parallelism
generator = CognitiveDataGenerator(ollama_client=ollama, max_parallel=8)

# Generate batch - automatically uses parallel processing
examples = generator.generate_batch(batch_size=100, model="llama3.2")
```

The `delay` parameter is now ignored as async handles rate limiting automatically!

### Option B: Install Ollama in Colab (Experimental)

⚠️ **Note:** This is experimental and may not work reliably in all Colab environments.

In [ ]:
# Uncomment and run this cell to install Ollama directly in Colab
# This is experimental and may not work in all environments

# !curl -fsSL https://ollama.ai/install.sh | sh
# 
# # Start Ollama in background
# import subprocess
# import time
# 
# # Start Ollama server
# ollama_process = subprocess.Popen(['ollama', 'serve'], 
#                                   stdout=subprocess.PIPE, 
#                                   stderr=subprocess.PIPE)
# 
# # Wait for server to start
# time.sleep(10)
# 
# # Pull a model
# !ollama pull llama3.2
# 
# print("Ollama installed and model pulled!")

## 4. Data Generation System

In [ ]:
# Import data generator module
from data_generator import *

print("Data generator loaded successfully!")
print("CognitiveDataGenerator class ready to use")

In [ ]:
# Verify all modules are imported and working
print("All modules imported successfully!")
print(f"Ready to generate data with {len(COGNITIVE_ACTIONS)} cognitive actions")
print(f"\nAvailable components:")
print(f"  - Variable pools: ✓")
print(f"  - Prompt templates: ✓")
print(f"  - Data generator: ✓")
print(f"  - Ollama client: {'✓' if 'ollama' in dir() else '⚠️ Configure in section 3'}")

## 5. Test the System

In [ ]:
# Test with Ollama (if connected)
# Configure max_parallel to control concurrency (default: 4, increase if you have more VRAM/RAM)
generator = CognitiveDataGenerator(ollama_client=ollama, max_parallel=4)

print("Testing single example generation...")

# Generate a single example
example = generator.generate_single_example(
    cognitive_action="reconsidering",
    template_type="single",
    model="gemma3:27b"  # Default model - change if you have a different model
)

if example:
    print("\n=== GENERATED EXAMPLE ===")
    print(f"Cognitive Action: {example.primary_cognitive_action}")
    print(f"Domain: {example.domain}")
    print(f"Complexity: {example.complexity}")
    print(f"Format: {example.format_type}")
    print()
    print("Generated Text:")
    print(example.text)
    print()
    print("Metadata:")
    for key, value in example.metadata.items():
        if key != 'prompt_used':  # Skip the long prompt
            print(f"  {key}: {value}")
else:
    print("Failed to generate example. Check Ollama connection.")

In [ ]:
# Test with Ollama (if connected)
# Configure max_parallel to control concurrency (default: 4, increase if you have more VRAM/RAM)
generator = CognitiveDataGenerator(ollama_client=ollama, max_parallel=4)

print("Testing single example generation...")

# Generate a single example
example = generator.generate_single_example(
    cognitive_action="reconsidering",
    template_type="single",
    model="llama3.2"  # Change this to your available model
)

if example:
    print("\n=== GENERATED EXAMPLE ===")
    print(f"Cognitive Action: {example.primary_cognitive_action}")
    print(f"Domain: {example.domain}")
    print(f"Complexity: {example.complexity}")
    print(f"Format: {example.format_type}")
    print()
    print("Generated Text:")
    print(example.text)
    print()
    print("Metadata:")
    for key, value in example.metadata.items():
        if key != 'prompt_used':  # Skip the long prompt
            print(f"  {key}: {value}")
else:
    print("Failed to generate example. Check Ollama connection.")

# Generate a small batch for testing with parallel processing
print("Generating small test batch using async parallel processing...")
print("This will be significantly faster than sequential generation!")

test_examples = generator.generate_batch(
    batch_size=10,
    cognitive_action=None,  # Random actions
    template_type="single",
    model="gemma3:27b"  # Default model
    # Note: 'delay' parameter is now ignored as we use async parallel processing
)

print(f"\n✓ Generated {len(test_examples)} examples")

# Show a few examples
for i, example in enumerate(test_examples[:3]):
    print(f"\n=== EXAMPLE {i+1} ===")
    print(f"Action: {example.primary_cognitive_action}")
    print(f"Domain: {example.domain}")
    print(f"Text: {example.text[:200]}...")

# Show statistics
generator.print_statistics()

In [ ]:
# Generate stratified dataset (smaller for testing) using parallel processing
print("Generating stratified test dataset with parallel processing...")
print("Note: Stratified generation uses multiple batch calls internally")

# Create new generator for clean stats with higher parallelism
# Increase max_parallel if you have sufficient VRAM/RAM (recommended: 8-16 for GPUs with 24GB+ VRAM)
stratified_generator = CognitiveDataGenerator(ollama_client=ollama, max_parallel=8)

stratified_examples = stratified_generator.generate_stratified_dataset(
    total_examples=100,  # Start small for testing
    model="gemma3:27b"  # Default model
)

print(f"\n✓ Generated {len(stratified_examples)} stratified examples")
stratified_generator.print_statistics()

In [ ]:
# Generate stratified dataset (smaller for testing) using parallel processing
print("Generating stratified test dataset with parallel processing...")
print("Note: Stratified generation uses multiple batch calls internally")

# Create new generator for clean stats with higher parallelism
# Increase max_parallel if you have sufficient VRAM/RAM (recommended: 8-16 for GPUs with 24GB+ VRAM)
stratified_generator = CognitiveDataGenerator(ollama_client=ollama, max_parallel=8)

stratified_examples = stratified_generator.generate_stratified_dataset(
    total_examples=100,  # Start small for testing
    model="llama3.2"
)

print(f"\n✓ Generated {len(stratified_examples)} stratified examples")
stratified_generator.print_statistics()

## 7. Data Analysis and Visualization

In [ ]:
# Convert to DataFrame for analysis
data = []
for example in stratified_generator.generated_examples:
    data.append({
        'text': example.text,
        'cognitive_action': example.primary_cognitive_action,
        'domain': example.domain,
        'complexity': example.complexity,
        'format_type': example.format_type,
        'text_length': len(example.text),
        'word_count': len(example.text.split()),
        'subject': example.metadata.get('subject', ''),
        'emotional_state': example.metadata.get('emotional_state', '')
    })

df = pd.DataFrame(data)
print(f"Dataset shape: {df.shape}")
print("\nFirst few rows:")
print(df.head())

print("\nBasic statistics:")
print(df.describe())

In [ ]:
# Visualizations
plt.figure(figsize=(15, 10))

# Distribution of cognitive actions
plt.subplot(2, 3, 1)
cognitive_action_counts = df['cognitive_action'].value_counts()
plt.bar(range(len(cognitive_action_counts)), cognitive_action_counts.values)
plt.xticks(range(len(cognitive_action_counts)), cognitive_action_counts.index, rotation=45, ha='right')
plt.title('Distribution of Cognitive Actions')
plt.ylabel('Count')

# Distribution of domains
plt.subplot(2, 3, 2)
domain_counts = df['domain'].value_counts()[:10]  # Top 10
plt.bar(range(len(domain_counts)), domain_counts.values)
plt.xticks(range(len(domain_counts)), domain_counts.index, rotation=45, ha='right')
plt.title('Top 10 Domains')
plt.ylabel('Count')

# Distribution of complexity levels
plt.subplot(2, 3, 3)
complexity_counts = df['complexity'].value_counts()
plt.pie(complexity_counts.values, labels=complexity_counts.index, autopct='%1.1f%%')
plt.title('Complexity Distribution')

# Text length distribution
plt.subplot(2, 3, 4)
plt.hist(df['text_length'], bins=20, edgecolor='black')
plt.title('Text Length Distribution')
plt.xlabel('Characters')
plt.ylabel('Frequency')

# Word count distribution
plt.subplot(2, 3, 5)
plt.hist(df['word_count'], bins=20, edgecolor='black')
plt.title('Word Count Distribution')
plt.xlabel('Words')
plt.ylabel('Frequency')

# Format type distribution
plt.subplot(2, 3, 6)
format_counts = df['format_type'].value_counts()
plt.pie(format_counts.values, labels=format_counts.index, autopct='%1.1f%%')
plt.title('Format Type Distribution')

plt.tight_layout()
plt.show()

# Show some sample texts by cognitive action
print("\n=== SAMPLE TEXTS BY COGNITIVE ACTION ===")
for action in df['cognitive_action'].unique()[:5]:  # First 5 actions
    sample = df[df['cognitive_action'] == action]['text'].iloc[0]
    print(f"\n{action.upper()}:")
    print(f"  {sample[:200]}...")

# Configuration for large-scale generation with parallel processing
# Target: 7,000 examples total (stratified across all cognitive actions)
# Optimized to fit within 5-7 hours of generation time
LARGE_SCALE_CONFIG = {
    'total_examples': 7000,   # Total target examples
    'model': 'gemma3:27b',    # Default model
    'max_parallel': 16,       # Parallel requests (uses 16 context buffers)
    'checkpoint_interval': 100,  # Save progress every 100 examples
    'checkpoint_dir': '/content/drive/MyDrive/cognitive_data_checkpoints'  # Google Drive location
}

print("Large scale configuration (with parallel processing):")
print(f"Total target examples: {LARGE_SCALE_CONFIG['total_examples']:,}")
print(f"Checkpoint interval: Every {LARGE_SCALE_CONFIG['checkpoint_interval']} examples")
print(f"Checkpoint directory: {LARGE_SCALE_CONFIG['checkpoint_dir']}")
print(f"Parallel requests: {LARGE_SCALE_CONFIG['max_parallel']} concurrent context buffers")

# Calculate stratified distribution
from variable_pools import COGNITIVE_ACTIONS
examples_per_action = LARGE_SCALE_CONFIG['total_examples'] // len(COGNITIVE_ACTIONS)
print(f"\nStratified distribution:")
print(f"  Cognitive actions: {len(COGNITIVE_ACTIONS)}")
print(f"  Examples per action: {examples_per_action:,}")

# With parallel processing, estimate time is significantly reduced
# Assuming 30 seconds per request average
estimated_seconds = LARGE_SCALE_CONFIG['total_examples'] / LARGE_SCALE_CONFIG['max_parallel'] * 30
estimated_hours = estimated_seconds / 3600
print(f"\nEstimated time (with {LARGE_SCALE_CONFIG['max_parallel']} parallel requests @ 30s each):")
print(f"  Total: {estimated_hours:.1f} hours ({estimated_hours*60:.0f} minutes)")
print(f"  vs sequential: {LARGE_SCALE_CONFIG['total_examples'] * 30 / 3600:.1f} hours - {LARGE_SCALE_CONFIG['max_parallel']}x speedup!")

print("\nTemplate distribution per action:")
print(f"  70% single action examples ({int(examples_per_action * 0.7)})")
print(f"  20% chain/combination examples ({int(examples_per_action * 0.2)})")
print(f"  5% dialogue examples ({int(examples_per_action * 0.05)})")
print(f"  5% thought-stream examples ({int(examples_per_action * 0.05)})")

print("\n" + "="*60)
print("VRAM Usage Expectations for 40GB GPU:")
print("="*60)
print(f"When IDLE (model loaded, no active requests):")
print(f"  • Model weights: ~18GB")
print(f"  • Current VRAM: ~18GB ✓ NORMAL")
print(f"\nDuring GENERATION ({LARGE_SCALE_CONFIG['max_parallel']} parallel requests):")
print(f"  • Model weights: ~18GB")
print(f"  • {LARGE_SCALE_CONFIG['max_parallel']} context buffers: ~{LARGE_SCALE_CONFIG['max_parallel'] * 1.5:.0f}GB")
print(f"  • Processing overhead: ~2GB")
estimated_vram = 18 + LARGE_SCALE_CONFIG['max_parallel'] * 1.5 + 2
print(f"  • Total during generation: ~{estimated_vram:.0f}GB")
print(f"  • Safety buffer: ~{40 - estimated_vram:.0f}GB")
print("\n💡 VRAM will jump from 18GB → ~38GB when you start generation!")
print("="*60)

In [ ]:
# Export the generated dataset
timestamp = int(time.time())

# Export as JSONL (recommended for large datasets)
jsonl_filename = f"cognitive_actions_dataset_{timestamp}.jsonl"
stratified_generator.export_dataset(jsonl_filename, format="jsonl")

# Export as JSON (for smaller datasets)
json_filename = f"cognitive_actions_dataset_{timestamp}.json"
stratified_generator.export_dataset(json_filename, format="json")

# Export as CSV for analysis
csv_filename = f"cognitive_actions_analysis_{timestamp}.csv"
df.to_csv(csv_filename, index=False)

print(f"\nDataset exported:")
print(f"  JSONL: {jsonl_filename}")
print(f"  JSON: {json_filename}")
print(f"  CSV: {csv_filename}")

# Show file sizes
import os
for filename in [jsonl_filename, json_filename, csv_filename]:
    if os.path.exists(filename):
        size = os.path.getsize(filename)
        print(f"  {filename}: {size:,} bytes ({size/1024:.1f} KB)")

# Configuration for large-scale generation with parallel processing
# Target: 7,000 examples total (stratified across all cognitive actions)
# Optimized to fit within 7-9 hours of generation time
LARGE_SCALE_CONFIG = {
    'total_examples': 7000,   # Total target examples (fits in ~7.3 hours @ 30s/request)
    'model': 'gemma3:27b',    # Default model
    'max_parallel': 6,        # Parallel requests (optimized for 40GB VRAM with 18GB model)
    'checkpoint_interval': 100,  # Save progress every 100 examples
    'checkpoint_dir': '/content/drive/MyDrive/cognitive_data_checkpoints'  # Google Drive location
}

print("Large scale configuration (with parallel processing):")
print(f"Total target examples: {LARGE_SCALE_CONFIG['total_examples']:,}")
print(f"Checkpoint interval: Every {LARGE_SCALE_CONFIG['checkpoint_interval']} examples")
print(f"Checkpoint directory: {LARGE_SCALE_CONFIG['checkpoint_dir']}")
print(f"Parallel requests: {LARGE_SCALE_CONFIG['max_parallel']} (optimized for 40GB VRAM)")

# Calculate stratified distribution
from variable_pools import COGNITIVE_ACTIONS
examples_per_action = LARGE_SCALE_CONFIG['total_examples'] // len(COGNITIVE_ACTIONS)
print(f"\nStratified distribution:")
print(f"  Cognitive actions: {len(COGNITIVE_ACTIONS)}")
print(f"  Examples per action: {examples_per_action:,}")

# With parallel processing, estimate time is significantly reduced
# Assuming 30 seconds per request average
estimated_seconds = LARGE_SCALE_CONFIG['total_examples'] / LARGE_SCALE_CONFIG['max_parallel'] * 30
estimated_hours = estimated_seconds / 3600
print(f"\nEstimated time (with {LARGE_SCALE_CONFIG['max_parallel']} parallel requests @ 30s each):")
print(f"  Total: {estimated_hours:.1f} hours ({estimated_hours*60:.0f} minutes)")
print(f"  vs sequential: {LARGE_SCALE_CONFIG['total_examples'] * 30 / 3600:.1f} hours - {LARGE_SCALE_CONFIG['max_parallel']}x speedup!")

print("\nTemplate distribution per action:")
print(f"  70% single action examples ({int(examples_per_action * 0.7)})")
print(f"  20% chain/combination examples ({int(examples_per_action * 0.2)})")
print(f"  5% dialogue examples ({int(examples_per_action * 0.05)})")
print(f"  5% thought-stream examples ({int(examples_per_action * 0.05)})")

print("\nVRAM estimation for 40GB GPU:")
print(f"  Model size: ~18GB")
print(f"  {LARGE_SCALE_CONFIG['max_parallel']} parallel requests (KV cache): ~{LARGE_SCALE_CONFIG['max_parallel'] * 2}GB")
print(f"  Processing overhead: ~2GB")
print(f"  Total estimated: ~{18 + LARGE_SCALE_CONFIG['max_parallel'] * 2 + 2}GB")
print(f"  Safety buffer: ~{40 - (18 + LARGE_SCALE_CONFIG['max_parallel'] * 2 + 2)}GB")

In [ ]:
# Uncomment and run this cell for large-scale stratified generation with parallel processing
# Target: 7,000 examples with checkpoints every 100 examples (fits in 7-9 hours)
# Uses stratified sampling to ensure all cognitive actions are represented evenly

# def run_large_scale_generation():
#     """Run stratified large-scale generation pipeline with parallel processing and checkpointing"""
#     
#     # Mount Google Drive for checkpoint storage (Colab only)
#     try:
#         from google.colab import drive
#         drive.mount('/content/drive')
#         print("✓ Google Drive mounted successfully")
#     except:
#         print("⚠️  Not running in Colab - checkpoints will be saved locally")
#         LARGE_SCALE_CONFIG['checkpoint_dir'] = './checkpoints'
#     
#     # Create checkpoint directory
#     import os
#     os.makedirs(LARGE_SCALE_CONFIG['checkpoint_dir'], exist_ok=True)
#     print(f"✓ Checkpoint directory created: {LARGE_SCALE_CONFIG['checkpoint_dir']}")
#     
#     # Initialize generator with parallel processing
#     large_generator = CognitiveDataGenerator(
#         ollama_client=ollama,
#         max_parallel=LARGE_SCALE_CONFIG['max_parallel']
#     )
#     
#     # Calculate examples per cognitive action for stratification
#     total_target = LARGE_SCALE_CONFIG['total_examples']
#     examples_per_action = total_target // len(COGNITIVE_ACTIONS)
#     checkpoint_interval = LARGE_SCALE_CONFIG['checkpoint_interval']
#     
#     print(f"\n{'='*60}")
#     print(f"🚀 Starting Stratified Generation")
#     print(f"Total target: {total_target:,} examples")
#     print(f"Cognitive actions: {len(COGNITIVE_ACTIONS)}")
#     print(f"Examples per action: {examples_per_action:,}")
#     print(f"Using {LARGE_SCALE_CONFIG['max_parallel']} parallel requests")
#     print(f"Checkpoints every {checkpoint_interval} examples")
#     print(f"{'='*60}\n")
#     
#     generation_start_time = time.time()
#     checkpoint_counter = 0
#     
#     # Generate stratified dataset by cognitive action
#     for action_idx, action in enumerate(COGNITIVE_ACTIONS.keys(), 1):
#         print(f"\n[{action_idx}/{len(COGNITIVE_ACTIONS)}] Generating {examples_per_action:,} examples for: {action}")
#         
#         # Mix template types for variety (70% single, 20% chain, 10% other)
#         template_distribution = (
#             ["single"] * int(examples_per_action * 0.7) +
#             ["chain"] * int(examples_per_action * 0.2) +
#             ["dialogue"] * int(examples_per_action * 0.05) +
#             ["thought_stream"] * int(examples_per_action * 0.05)
#         )
#         # Adjust to exact count
#         while len(template_distribution) < examples_per_action:
#             template_distribution.append("single")
#         template_distribution = template_distribution[:examples_per_action]
#         random.shuffle(template_distribution)
#         
#         # Generate in checkpoint-sized batches
#         num_batches = (examples_per_action + checkpoint_interval - 1) // checkpoint_interval
#         
#         for batch_idx in range(num_batches):
#             start_idx = batch_idx * checkpoint_interval
#             end_idx = min(start_idx + checkpoint_interval, examples_per_action)
#             batch_templates = template_distribution[start_idx:end_idx]
#             
#             # Count template types in this batch
#             template_counts = {}
#             for template in batch_templates:
#                 template_counts[template] = template_counts.get(template, 0) + 1
#             
#             # Generate for each template type in batch
#             batch_examples = []
#             for template_type, count in template_counts.items():
#                 print(f"  Batch {batch_idx + 1}/{num_batches}: Generating {count} {template_type} examples...")
#                 examples = large_generator.generate_batch(
#                     batch_size=count,
#                     cognitive_action=action,
#                     template_type=template_type,
#                     model=LARGE_SCALE_CONFIG['model']
#                 )
#                 batch_examples.extend(examples)
#             
#             # Save checkpoint
#             checkpoint_counter += 1
#             checkpoint_filename = os.path.join(
#                 LARGE_SCALE_CONFIG['checkpoint_dir'],
#                 f"checkpoint_{checkpoint_counter:04d}_{action}_{int(time.time())}.jsonl"
#             )
#             
#             with open(checkpoint_filename, 'w') as f:
#                 for example in batch_examples:
#                     json_obj = {
#                         'text': example.text,
#                         'primary_cognitive_action': example.primary_cognitive_action,
#                         'secondary_actions': example.secondary_actions,
#                         'domain': example.domain,
#                         'complexity': example.complexity,
#                         'perspective': example.perspective,
#                         'format_type': example.format_type,
#                         'metadata': example.metadata
#                     }
#                     f.write(json.dumps(json_obj) + '\n')
#             
#             print(f"  ✓ Checkpoint {checkpoint_counter} saved: {os.path.basename(checkpoint_filename)}")
#             print(f"  Total progress: {len(large_generator.generated_examples):,}/{total_target:,} ({len(large_generator.generated_examples)/total_target*100:.1f}%)\n")
#     
#     # Final export
#     elapsed_time = time.time() - generation_start_time
#     final_timestamp = int(time.time())
#     final_filename = os.path.join(
#         LARGE_SCALE_CONFIG['checkpoint_dir'],
#         f"cognitive_actions_stratified_7k_{final_timestamp}.jsonl"
#     )
#     large_generator.export_dataset(final_filename, format="jsonl")
#     
#     print(f"\n{'='*60}")
#     print(f"🎉 COMPLETE! Generated {len(large_generator.generated_examples):,} examples")
#     print(f"⏱️  Total time: {elapsed_time/3600:.2f} hours ({elapsed_time/60:.1f} minutes)")
#     print(f"📁 Final dataset: {final_filename}")
#     print(f"📊 Total checkpoints: {checkpoint_counter}")
#     print(f"📊 Total errors: {len(large_generator.generation_stats['errors'])}")
#     print(f"{'='*60}\n")
#     
#     # Print final statistics
#     large_generator.print_statistics()
#     
#     return large_generator

# # Uncomment the line below to start large-scale generation
# # large_generator = run_large_scale_generation()

print("Stratified large-scale generation function defined with parallel processing.")
print(f"Target: {LARGE_SCALE_CONFIG['total_examples']:,} examples with checkpoints every {LARGE_SCALE_CONFIG['checkpoint_interval']} examples")
print(f"Distribution: ~{LARGE_SCALE_CONFIG['total_examples'] // len(COGNITIVE_ACTIONS):,} examples per cognitive action")
print("\nTo start generation:")
print("1. Uncomment the last line: large_generator = run_large_scale_generation()")
print("2. Run this cell")
print("3. Checkpoints will be saved to Google Drive (if running in Colab)")
estimated_hours = LARGE_SCALE_CONFIG['total_examples'] / LARGE_SCALE_CONFIG['max_parallel'] * 30 / 3600
print(f"\n⏱️  Estimated time: ~{estimated_hours:.1f} hours ({estimated_hours*60:.0f} minutes) @ 30s per request")

In [ ]:
# Uncomment and run this cell for large-scale generation with parallel processing
# WARNING: This will take a very long time, but much faster than sequential!

# def run_large_scale_generation():
#     """Run the complete large-scale generation pipeline with parallel processing"""
#     
#     # Initialize generator with parallel processing
#     large_generator = CognitiveDataGenerator(
#         ollama_client=ollama,
#         max_parallel=LARGE_SCALE_CONFIG['max_parallel']
#     )
#     
#     phases = [
#         ('phase1_round1', 'single', LARGE_SCALE_CONFIG['phase1_round1']),
#         ('phase1_round2', 'chain', LARGE_SCALE_CONFIG['phase1_round2']),
#         ('phase2', 'single', LARGE_SCALE_CONFIG['phase2']),
#         ('phase3', 'single', LARGE_SCALE_CONFIG['phase3']),
#         ('phase4', 'negative', LARGE_SCALE_CONFIG['phase4']),
#         ('phase5', 'dialogue', LARGE_SCALE_CONFIG['phase5']),
#         ('phase6', 'thought_stream', LARGE_SCALE_CONFIG['phase6'])
#     ]
#     
#     for phase_name, template_type, target_count in phases:
#         print(f"\n🚀 Starting {phase_name}: {target_count:,} examples")
#         print(f"Template type: {template_type}")
#         print(f"Using {LARGE_SCALE_CONFIG['max_parallel']} parallel requests")
#         
#         phase_examples = large_generator.generate_batch(
#             batch_size=target_count,
#             template_type=template_type,
#             model=LARGE_SCALE_CONFIG['model']
#             # No delay needed with async parallel processing!
#         )
#         
#         print(f"✅ Completed {phase_name}: {len(phase_examples):,} examples")
#         
#         # Checkpoint save
#         checkpoint_filename = f"checkpoint_{phase_name}_{int(time.time())}.jsonl"
#         large_generator.export_dataset(checkpoint_filename, format="jsonl")
#         print(f"💾 Checkpoint saved: {checkpoint_filename}")
#         
#         # Print progress statistics
#         large_generator.print_statistics()
#     
#     # Final export
#     final_timestamp = int(time.time())
#     final_filename = f"cognitive_actions_complete_dataset_{final_timestamp}.jsonl"
#     large_generator.export_dataset(final_filename, format="jsonl")
#     
#     print(f"\n🎉 COMPLETE! Generated {len(large_generator.generated_examples):,} examples")
#     print(f"📁 Final dataset: {final_filename}")
#     
#     return large_generator

# # Uncomment the line below to start large-scale generation
# # large_generator = run_large_scale_generation()

print("Large-scale generation function defined with parallel processing.")
print("Uncomment the last line and run this cell to start large-scale generation.")
print(f"⚠️  With {LARGE_SCALE_CONFIG['max_parallel']}x parallel processing, this will be {LARGE_SCALE_CONFIG['max_parallel']}x faster!")
print("Make sure you have sufficient time and resources before starting!")

## 10. Quality Control and Validation

In [ ]:
def quality_control_analysis(examples):
    """Perform quality control analysis on generated examples"""
    
    print("=== QUALITY CONTROL ANALYSIS ===")
    
    # Basic statistics
    total_examples = len(examples)
    print(f"Total examples analyzed: {total_examples:,}")
    
    # Text length analysis
    text_lengths = [len(ex.text) for ex in examples]
    word_counts = [len(ex.text.split()) for ex in examples]
    
    print(f"\nText Length Statistics:")
    print(f"  Average characters: {np.mean(text_lengths):.1f}")
    print(f"  Average words: {np.mean(word_counts):.1f}")
    print(f"  Min/Max characters: {min(text_lengths)} / {max(text_lengths)}")
    print(f"  Min/Max words: {min(word_counts)} / {max(word_counts)}")
    
    # Coverage analysis
    cognitive_actions = set(ex.primary_cognitive_action for ex in examples)
    domains = set(ex.domain for ex in examples)
    
    print(f"\nCoverage Statistics:")
    print(f"  Cognitive actions covered: {len(cognitive_actions)}/{len(COGNITIVE_ACTIONS)} ({len(cognitive_actions)/len(COGNITIVE_ACTIONS)*100:.1f}%)")
    print(f"  Domains covered: {len(domains)}/{len(DOMAINS)} ({len(domains)/len(DOMAINS)*100:.1f}%)")
    
    # Quality indicators
    print(f"\nQuality Indicators:")
    
    # Check for very short examples
    very_short = sum(1 for length in text_lengths if length < 50)
    print(f"  Very short examples (<50 chars): {very_short} ({very_short/total_examples*100:.1f}%)")
    
    # Check for very long examples
    very_long = sum(1 for length in text_lengths if length > 1000)
    print(f"  Very long examples (>1000 chars): {very_long} ({very_long/total_examples*100:.1f}%)")
    
    # Check for examples that might contain prompt artifacts
    prompt_artifacts = sum(1 for ex in examples if any(word in ex.text.lower() for word in 
                          ['generate', 'example', 'sentence', 'requirement', 'output']))
    print(f"  Potential prompt artifacts: {prompt_artifacts} ({prompt_artifacts/total_examples*100:.1f}%)")
    
    # Diversity check - look for repeated phrases
    all_texts = [ex.text for ex in examples]
    unique_texts = set(all_texts)
    print(f"  Unique texts: {len(unique_texts)}/{total_examples} ({len(unique_texts)/total_examples*100:.1f}%)")
    
    return {
        'total_examples': total_examples,
        'avg_length': np.mean(text_lengths),
        'avg_words': np.mean(word_counts),
        'cognitive_actions_covered': len(cognitive_actions),
        'domains_covered': len(domains),
        'very_short_pct': very_short/total_examples*100,
        'very_long_pct': very_long/total_examples*100,
        'prompt_artifacts_pct': prompt_artifacts/total_examples*100,
        'uniqueness_pct': len(unique_texts)/total_examples*100
    }

# Run quality control on current examples
if stratified_generator.generated_examples:
    qc_results = quality_control_analysis(stratified_generator.generated_examples)
else:
    print("No examples to analyze. Generate some examples first.")

In [ ]:
# Manual review of sample examples
def review_samples(examples, n_samples=5):
    """Review a random sample of examples for quality"""
    
    if not examples:
        print("No examples to review.")
        return
    
    print("=== MANUAL QUALITY REVIEW ===")
    print(f"Reviewing {min(n_samples, len(examples))} random examples:\n")
    
    sample_examples = random.sample(examples, min(n_samples, len(examples)))
    
    for i, example in enumerate(sample_examples, 1):
        print(f"--- EXAMPLE {i} ---")
        print(f"Cognitive Action: {example.primary_cognitive_action}")
        print(f"Domain: {example.domain}")
        print(f"Complexity: {example.complexity}")
        print(f"Format: {example.format_type}")
        print(f"Length: {len(example.text)} chars, {len(example.text.split())} words")
        print()
        print("Text:")
        print(f"\"{example.text}\"")
        print()
        print("Subject:", example.metadata.get('subject', 'N/A'))
        print("Emotional State:", example.metadata.get('emotional_state', 'N/A'))
        print("Unique Angle:", example.metadata.get('unique_angle', 'N/A'))
        print("\n" + "="*60 + "\n")

# Review samples
if stratified_generator.generated_examples:
    review_samples(stratified_generator.generated_examples, n_samples=3)
else:
    print("No examples to review. Generate some examples first.")

## 🎉 Conclusion

You now have a complete cognitive action data generation system!

### What You've Built:
- **Scientific Foundation**: Based on established taxonomies from cognitive psychology
- **Flexible Architecture**: Modular system with variable pools and templates
- **Scalable Generation**: Can generate from small batches to 100,000+ examples
- **Quality Control**: Built-in analysis and validation tools
- **Multiple Formats**: Support for various example types (single actions, chains, dialogues, thought streams)

### Next Steps:
1. **Scale Up**: Use the large-scale generation for your full dataset
2. **Fine-tune**: Adjust templates and variables based on your specific needs
3. **Validate**: Run quality control on larger datasets
4. **Train Models**: Use the generated data to train your cognitive action recognition models

### Tips for Production Use:
- Monitor generation quality regularly
- Save checkpoints during long generation runs
- Experiment with different Ollama models for variety
- Consider post-processing for consistency

**Happy data generating! 🚀**